In [29]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import numpy as np
import os

import time

In [30]:
country = "BR"
geo1_nq = "BR_estado"
geo2_nq = "BR_cidade"
geo1_ipums = 'GEO1_BR2010'
geo2_ipums = 'GEO2_BR2010'
year = '2010'

In [31]:
census = pd.read_csv("../Data/"+country+"/sample/"+"ipumsi_00026.csv")

In [32]:
netquest = pd.read_csv("../Data/"+country+"/panel/"+country+"_netquest-panel.csv")

In [33]:
ipums_geo2 = pd.read_csv('../Data/'+country+'/sample/ipums_codebook_GEO2_BR2010.csv',encoding='Latin1', names=['code','name'], skiprows=1)
ipums_geo1 = pd.read_csv('../Data/'+country+'/sample/ipums_codebook_GEO1_BR2010.csv',encoding='Latin1', names=['code','name'], skiprows=1)

In [34]:
ipums_geodf = census[[geo1_ipums,geo2_ipums]]
ipums_geodf.columns = ['geo1_code','geo2_code']
ipums_geodf.drop_duplicates(inplace=True)

ipums_geodf['geo1_name'] = ipums_geodf.merge(ipums_geo1, 
                                             how='left', 
                                             left_on = "geo1_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

ipums_geodf['geo2_name'] = ipums_geodf.merge(ipums_geo2, 
                                             how='left', 
                                             left_on = "geo2_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

We have to get the melting action going here.

In [35]:
ipums_geodf = ipums_geodf.geo2_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo2_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo2_name')], value_name = "geo2_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo2_name'])
ipums_geodf.geo2_name = ipums_geodf.geo2_name.str.strip()

In [36]:
ipums_geodf = ipums_geodf.geo1_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo1_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo1_name')], value_name = "geo1_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo1_name'])
ipums_geodf.geo1_name = ipums_geodf.geo1_name.str.strip()

Now to Netquest.

In [37]:
# province_labels = pd.read_excel("../Data/CL/provincia_dict.xlsx", encoding='Latin1', header=None)
nq_geodf = netquest[[geo1_nq,geo1_nq+'_name',geo2_nq,geo2_nq+'_name']]
nq_geodf.columns = ['geo1_code','geo1_name','geo2_code','geo2_name']

nq_geodf.drop_duplicates(inplace=True)

C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Finding the duplicates in nq_geodf.geo2_code, and keeping the combinations that occur most often (which are quite clearly the good ones).

In [38]:
nq_geodf['count'] = nq_geodf.apply(lambda r: sum((netquest[geo1_nq]==r['geo1_code'])&(netquest[geo2_nq]==r['geo2_code'])),
                                   axis=1)
# nq_geodf.sort_values(['geo2_code','count']) \
#     [(nq_geodf.sort_values(['geo2_code','count']).duplicated('geo2_code',keep=False))]
nq_geodf = nq_geodf.sort_values(['geo2_code','count']) \
    [~(nq_geodf.sort_values(['geo2_code','count']).duplicated('geo2_code',keep='last'))]

C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Manual fixes, as found necessary in the next step...

In [39]:
ipums_geodf.loc[ipums_geodf.geo1_name.str.contains("Federal District"),
                'geo1_name'] = "Distrito Federal"


ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("Santarém"))& \
                (ipums_geodf.geo1_name.str.contains("Paraíba")),
                'geo2_name'] = "Joca Claudino"

ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("Presidente Juscelino"))& \
                (ipums_geodf.geo1_name.str.contains("Rio Grande do Norte")),
                'geo2_name'] = "Serra Caiada"

nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Bonfim"))& \
                (nq_geodf.geo1_name.str.contains("Rio de Janeiro")),
                'geo1_name'] = "Minas Gerais"

nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Iracema"))& \
                (nq_geodf.geo1_name.str.contains("Santa Catarina")),
                'geo2_name'] = "Itaiópolis"


nq_geodf.loc[nq_geodf.geo2_name=="Pescaria Brava",'geo2_name'] = 'Laguna'
nq_geodf.loc[nq_geodf.geo2_name=="Balneário Rincão",'geo2_name'] = 'Içara'
nq_geodf.loc[nq_geodf.geo2_name=="Pinto Bandeira",'geo2_name'] = 'Bento Gonçalves'
nq_geodf.loc[nq_geodf.geo2_name=="Paraíso das Águas",'geo2_name'] = 'Água Clara'
nq_geodf.loc[nq_geodf.geo2_name=="Açailândia",'geo1_name'] = 'Maranhão'
nq_geodf.loc[nq_geodf.geo2_name=="Mariana",'geo1_name'] = 'Minas Gerais'
nq_geodf.loc[nq_geodf.geo2_name=="Fortaleza",'geo1_name'] = 'Ceará'
nq_geodf.loc[nq_geodf.geo2_name.isin(["Leme",
                                      "São Paulo",
                                      'Itapevi',
                                      "Franca",
                                      "Presidente Prudente",
                                      "Suzano",
                                      "Ourinhos"
                                     ]),'geo1_name'] = 'São Paulo'

nq_geodf.loc[(nq_geodf.geo2_name=="Praia Grande") &\
             ((nq_geodf.geo1_name=="Acre")|(nq_geodf.geo1_name=="Maranhão")),
             'geo1_name'] = 'São Paulo'

nq_geodf.loc[nq_geodf.geo1_name=="Brasília",'geo1_name'] = 'Distrito Federal'
nq_geodf.loc[nq_geodf.geo2_name=="Brasília",'geo1_name'] = 'Distrito Federal'
nq_geodf.loc[nq_geodf.geo2_name=="Tupanatinga",'geo1_name'] = 'Pernambuco'
nq_geodf.loc[nq_geodf.geo2_name=="Macaé",'geo1_name'] = 'Rio de Janeiro'
nq_geodf.loc[nq_geodf.geo2_name=="Queimados",'geo1_name'] = 'Rio de Janeiro'
nq_geodf.loc[nq_geodf.geo2_name=="São Miguel do Iguaçu",'geo1_name'] = 'Paraná'



Next, a function for fuzzy-joining columns from dataframes

In [40]:
def fuzzy_join(df1, df2, varname, subset=False, cutoff=90):
    '''Returns a dataframe the length of df1, with the matches on the given variable
    
        Assumes the var is named the same in both dataframes.
        
        subset is a tuple or list of length 2 whose first element is a string indicating which
        variable in df1 has to be equal to which variable in df2 (specified by 2nd element)
        '''
    #figure out if there is a constraint
    if not subset:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)
        
    else:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)    

    
    # different match cases
    #morRatioMatch = ratio_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    oneRatioMatch = ratio_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noRatioMatch = ratio_matches.apply(lambda l: (l[0][1]<cutoff))

    #morPartiMatch = parti_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    onePartiMatch = parti_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noPartiMatch = parti_matches.apply(lambda l: (l[0][1]<cutoff))
    
    # pick out what's better
    matches = pd.Series([(np.nan,np.nan,np.nan)]*len(df1),index=df1.index)
    matches.loc[oneRatioMatch] = ratio_matches.loc[oneRatioMatch].apply(lambda l: l[0])
    matches.loc[(noRatioMatch&onePartiMatch)] = parti_matches.loc[(noRatioMatch&onePartiMatch)]\
        .apply(lambda l: l[0])
    matches.loc[(noRatioMatch&noPartiMatch)] = parti_matches.loc[(noRatioMatch&noPartiMatch)]\
        .apply(lambda l: l[0])
    
    df = pd.DataFrame()
    df['name'] = matches.apply(lambda l: l[0])
    df['score'] = matches.apply(lambda l: l[1])
    df['index'] = matches.apply(lambda l: l[2])
    df['parti_matches'] = parti_matches
    df['ratio_matches'] = ratio_matches
    
    return df

Use the new tool!

In [41]:
nq_geodf[['geo1_match_name','geo1_match_score','geo1_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo1_name')[['name','score','index']]

In [42]:
nq_geodf[['geo2_match_name','geo2_match_score','geo2_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo2_name', 
             subset=['geo1_match_name','geo1_name']
            )[['name','score','index']]

## De-duping

What can we eliminate? Where geo_2code is duplicated and...
* where geo2_code and geo2_match_code are duplicated across dupes, just de-dupe--doesn't matter which we use
* drop any that have NaN in their geo-s
* one of the geo2_match_names is an exact match, throw out any others
* if there are two contenders, check which code+geo1_name+geo2_name combo has the majority

In [43]:
tmp = nq_geodf.copy()

# # if we have the same mapping between netquest geo2 and ipums geo2, doesn't matter which we pick
# tmp = tmp.drop_duplicates(subset=['geo2_code','geo2_match_index'])
# # ones that have NA geographies are useless in this context, so drop them first
# tmp = tmp[(~tmp.geo1_code.isna())& (~tmp.geo2_code.isna())]

# dupes = tmp[tmp.duplicated('geo2_code')].geo2_code
# for code in dupes: 
#     # if one of the geo2_match_names is an exact match, throw out any others
#     if sum(tmp.loc[tmp.geo2_code==code,"geo2_name"]==tmp.loc[tmp.geo2_code==code,"geo2_match_name"])>0:
#         tmp = tmp[~((tmp.geo2_code==code) & (tmp.geo2_name!=tmp.geo2_match_name))]
#     # if there are still duplicates, use the one that has the most prevalent pairing of geo1 & geo2
#     id_of_most_common = tmp.loc[tmp.geo2_code==code,"geo1_code"].apply(
#         lambda c: sum((netquest[geo1_nq]==c) & (netquest[geo2_nq]==code))
#     ).idxmax()
#     correct_geo1 = tmp.loc[id_of_most_common,"geo1_code"]
#     tmp = tmp[~((tmp.geo2_code==code) & ~(tmp.geo1_code==correct_geo1))]
        


**What is still an issue??**

In [44]:
tmp[tmp.duplicated('geo2_code', keep=False)].sort_values('geo2_code')#\
# [["geo2_code","geo1_name","geo1_match_name","geo2_name","geo2_match_name","geo2_match_score","geo2_match_index"]]

,geo1_code,geo1_name,geo2_code,geo2_name,count,geo1_match_name,geo1_match_score,geo1_match_index,geo2_match_name,geo2_match_score,geo2_match_index


Success! This can be the geo-df.

In [45]:
nq_geodf = tmp

In [46]:
nq_geodf[nq_geodf.geo2_match_score<90][['geo1_name',
                                        'geo1_match_name',
                                        'geo2_name',
                                        'geo2_match_name',
                                        'geo2_match_score']
                                      ].sort_values('geo1_name'
                                      )

,geo1_name,geo1_match_name,geo2_name,geo2_match_name,geo2_match_score
274193,Paraíba,Paraíba,Tácima,Tacima,83


Get a crosswalk for these!

In [47]:
has_ipums_geo = nq_geodf.geo2_match_index.notna()

In [48]:
nq_geodf['IPUMS_geo2_code'] = np.nan

nq_geodf.loc[has_ipums_geo,'IPUMS_geo2_code'] = nq_geodf[has_ipums_geo]\
                            .geo2_match_index\
                            .astype(int)\
                            .apply(
                                lambda i: ipums_geodf.loc[i,'geo2_code']
                            .astype(int)
)

We want to replace each nq_geo2 with the _code_ for the corresponding census geography.



Read in the municipio-centroids, and hopefully attach them here.

In [49]:
geo2_centroids = pd.read_csv('../Data/'+country+'/geography/'+country+'_geo_centroids.csv', encoding='latin1')

In [50]:
nq_geodf_merged = nq_geodf.merge(geo2_centroids[['ADMIN_NAME','Y','X','IPUM'+year]], 
               left_on='IPUMS_geo2_code',
               right_on="IPUM"+year,
               how='left'
              ).drop('IPUM'+year, axis=1)

In [51]:
panel_geo = netquest.merge(nq_geodf_merged[['X','Y','geo2_code']],
               left_on=geo2_nq,
               right_on='geo2_code',
               how='left'
              ).drop('geo2_code',axis=1)

In [52]:
panel_geo.shape[0]==netquest.shape[0]

True

In [53]:
panel_geo.to_csv('../Data/'+country+'/panel/'+country+'_netquest-panel_geo.csv')

And the same for census: attach geographies

In [54]:
census_geo = census.merge(geo2_centroids[['ADMIN_NAME','X',"Y",'IPUM'+year]],
                          left_on = geo2_ipums,
                          right_on='IPUM'+year,
                          how='left'
                         ).drop('IPUM'+year,axis=1)

In [55]:
census_geo.shape[0]==census.shape[0]

True

In [56]:
census_geo.to_csv('../Data/'+country+'/sample/'+country+'_ipums-census_geo.csv')